In [1]:
from glob import glob
from tensorflow import keras

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
IMAGE_SIZE = [150,150]

In [3]:
# Dataset can be downloaded from https://www.kaggle.com/datasets/apollo2506/facial-recognition-dataset/data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_dir = '/content/drive/MyDrive/Datasets/Training/Training'
test_dir = '/content/drive/MyDrive/Datasets/Testing/Testing'

train_angry_dir = '/content/drive/MyDrive/Datasets/Training/Training/Angry'
train_fear_dir = '/content/drive/MyDrive/Datasets/Training/Training/Fear'
train_happy_dir = '/content/drive/MyDrive/Datasets/Training/Training/Happy'
train_neutral_dir = '/content/drive/MyDrive/Datasets/Training/Training/Neutral'
train_sad_dir = '/content/drive/MyDrive/Datasets/Training/Training/Sad'
train_surprise_dir = '/content/drive/MyDrive/Datasets/Training/Training/Suprise'

test_angry_dir = '/content/drive/MyDrive/Datasets/Testing/Testing/Angry'
test_fear_dir = '/content/drive/MyDrive/Datasets/Testing/Testing/Fear'
test_happy_dir = '/content/drive/MyDrive/Datasets/Testing/Testing/Happy'
test_neural_dir = '/content/drive/MyDrive/Datasets/Testing/Testing/Neutral'
test_sad_dir = '/content/drive/MyDrive/Datasets/Testing/Testing/Sad'
test_surprise_dir = '/content/drive/MyDrive/Datasets/Testing/Testing/Suprise'

dir_list = [train_angry_dir, train_fear_dir, train_happy_dir,
           train_neutral_dir, train_sad_dir, train_surprise_dir,
           test_angry_dir, test_fear_dir, test_happy_dir,
           test_neural_dir, test_sad_dir, test_surprise_dir]

In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
for layer in vgg.layers:
    # don't train already trained layers
    layer.trainable=False

In [7]:
folders = glob('/content/drive/MyDrive/Datasets/Training/Training/*')
print(len(folders))

6


In [8]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders),activation='softmax')(x)
model = Model(inputs=vgg.input,outputs=prediction)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
train_loader = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=False)
test_loader = ImageDataGenerator(rescale=1./255)

In [11]:
training_set = train_loader.flow_from_directory(train_dir,target_size=(150,150),batch_size=64,class_mode='categorical')
test_set = test_loader.flow_from_directory(test_dir,target_size=(150,150),batch_size=62,class_mode='categorical')

Found 28273 images belonging to 6 classes.
Found 7067 images belonging to 6 classes.


In [12]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,
                                                 restore_best_weights=True)

In [13]:
history = model.fit_generator(training_set,
                   epochs=1,
                   verbose=1,
                   validation_data=test_set,
                   callbacks=early_stopping_cb)

<ipython-input-13-5d6e678e0ea9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(training_set,


442/442 [==============================] - 8504s 19s/step - loss: 1.4246 - accuracy: 0.4435 - val_loss: 1.3424 - val_accuracy: 0.4770


In [14]:
model.save('facial_recog_vgg16.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
